In [5]:
import re, os
import bs4
import pandas as pd
from tqdm import tqdm_notebook
import urllib
import numpy as np

# Ingesting the Index #

In [6]:
SVO_BASE_URL = 'http://svo2.cab.inta-csic.es/svo/theory/fps'

In [7]:
def generate_facilities_list():
    svo_html = urllib2.urlopen(SVO_BASE_URL).read()
    svo_soup = bs4.BeautifulSoup(svo_html)
    observatory_links = svo_soup.findAll('a', attrs={'href':re.compile('gname')})
    observatory_list = pd.DataFrame(index=range(len(observatory_links)), columns=['facility', 'url'])
    for i, obs_link in enumerate(observatory_links):
        observatory_list.loc[i] = obs_link.text, obs_link.attrs['href']
    return observatory_list

def generate_instruments_list(observatory_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    return observatory_soup.findAll('a', attrs={'href':re.compile('index\.php\?mode=browse&gname=.+?&gname2=.+?')})
#    filter_soup = observatory_soup.findAll('table')[8]
#    filter_table = pd.read_html(str(filter_soup), header=0)[0]
#    return filter_table

def generate_filter_list(instrument_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    filter_soup = observatory_soup.findAll('table')[8]
    filter_table = pd.read_html(str(filter_soup), header=0)[0]
    return filter_table

In [8]:
facilities_list = generate_facilities_list()

NameError: name 'urllib2' is not defined

In [10]:
all_filters = None
for i, facility in tqdm_notebook(facilities_list.iterrows(), desc='Facilities', total=len(facilities_list)):
    print("Working on {0}".format(facility.facility))
    instruments = generate_instruments_list(os.path.join(SVO_BASE_URL, facility.url))
    if u'\xc2' in facility.facility:
        1/0
    if len(instruments) == 0:
        #print "{0} does not have separate instruments".format(facility.facility)
        filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, facility.url))
        filter_list['Obs. Facility'] =  facility.facility
        filter_list['Instrument'] =  'NA'
        #filter_list['']
        if all_filters is None:
            all_filters = filter_list
        else:
            all_filters = all_filters.append(filter_list)
    else:
        for instrument in tqdm_notebook(instruments, desc='Instruments', leave=False):
            #print "Working on {0}".format(instrument.text)
            filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, instrument.attrs['href']))
            filter_list['Obs. Facility'] =  facility.facility
            filter_list['Instrument'] =  instrument.text

            if all_filters is None:
                all_filters = filter_list
            else:
                all_filters = all_filters.append(filter_list)


NameError: name 'facilities_list' is not defined

In [135]:
all_filters.reset_index(inplace=True, drop=True)

In [136]:
columns = all_filters.columns.copy().tolist()
columns[1:5] = ['lambda_' + item[2:] for item in all_filters.columns[1:5]] 
all_filters.columns = columns

In [137]:
all_filters.to_hdf('svo_filter_index.h5', 'all_filters')

In [139]:
all_filters[all_filters['Obs. Facility'] == 'Geneva']

,Filter ID,lambda_mean,lambda_eff,lambda_min,lambda_max,Weff,ZP (Jy),Obs. Facility,Instrument,Description
1559,Geneva/Geneva.U,3459.1,3460.5,3006,3892,484.9,1307.8,Geneva,NA,Geneva U
1560,Geneva/Geneva.B1,4022.8,4036.4,3604,4669,422.1,3699.4,Geneva,NA,Geneva B1
1561,Geneva/Geneva.B,4248.1,4235.3,3609,5102,787.9,3953.8,Geneva,NA,Geneva B
1562,Geneva/Geneva.B2,4480.7,4467.5,4023,5012,437.8,4177.5,Geneva,NA,Geneva B2
1563,Geneva/Geneva.V2,5409.5,5387.3,5003,6105,498.6,3654.4,Geneva,NA,Geneva V2
1564,Geneva/Geneva.V,5508.5,5462.1,4910,6502,749.7,3591.8,Geneva,NA,Geneva V
1565,Geneva/Geneva.G,5814.6,5792.8,5404,6547,476.1,3407.6,Geneva,NA,Geneva G


# Downloading the transmission curves #

In [141]:
tab = pd.read_hdf('svo_filter_index.h5')

In [146]:
ls SVO/

u'SVO/DENIS/DENIS.I'

In [149]:
filter_data

Filter ID                                GCPD/Johnson.R_Mendoza
lambda_mean                                              6936.1
lambda_eff                                               6694.1
lambda_min                                                 5250
lambda_max                                                 9450
Weff                                                     2071.5
ZP (Jy)                                                  2816.2
Obs. Facility                                              GCPD
Instrument                                              Johnson
Description      Johnson R filter in GCPD-UBVRI, Mendoza (ADPS)
Name: 1324, dtype: object

In [150]:
for i, filter_data in tqdm_notebook(tab.iterrows()):
    filt_id = filter_data['Filter ID']
    facility = filter_data['Obs. Facility']
    if not os.path.exists(os.path.join('SVO', facility)):
        os.makedirs(os.path.join('SVO', facility))
    filename = filt_id.split('/')
    outfile = os.path.join('SVO', facility, filename[-1])
    if os.path.exists(outfile):
        continue
    url = 'http://svo2.cab.inta-csic.es/svo/theory/fps/getdata.php?format=ascii&id=' + filt_id
    print(outfile)
    urllib.urlretrieve(url, outfile)


SVO/GCPD/Johnson.R_Mendoza
SVO/GCPD/Johnson.R
SVO/GCPD/Johnson.I_Landolt
SVO/GCPD/Johnson.I_Mendoza
SVO/GCPD/Stromgren.u_ADPS
SVO/GCPD/Stromgren.u
SVO/GCPD/Stromgren.v_ADPS
SVO/GCPD/Stromgren.v
SVO/GCPD/Stromgren.b_ADPS
SVO/GCPD/Stromgren.b
SVO/GCPD/Stromgren.y
SVO/GCPD/Stromgren.y_ADPS
SVO/GCPD/Vilnius.U_ADPS
SVO/GCPD/Vilnius.U
SVO/GCPD/Vilnius.P_ADPS
SVO/GCPD/Vilnius.P
SVO/GCPD/Vilnius.X_ADPS
SVO/GCPD/Vilnius.X
SVO/GCPD/Vilnius.Y_ADPS
SVO/GCPD/Vilnius.Y
SVO/GCPD/Vilnius.Z_ADPS
SVO/GCPD/Vilnius.Z
SVO/GCPD/Vilnius.V_ADPS
SVO/GCPD/Vilnius.V
SVO/GCPD/Vilnius.S
SVO/GCPD/Vilnius.S_ADPS
SVO/GCPD/Walraven.W
SVO/GCPD/Walraven.U
SVO/GCPD/Walraven.L
SVO/GCPD/Walraven.B
SVO/GCPD/Walraven.V
SVO/GCPD/Washington.C_ph
SVO/GCPD/Washington.C_pe
SVO/GCPD/Washington.M_pe
SVO/GCPD/Washington.M_ph
SVO/GCPD/Washington.T1_pe
SVO/GCPD/Washington.T1_ph
SVO/GCPD/Washington.T2_pe
SVO/GCPD/Washington.T2_ph
SVO/GCPD/WBVR.W_ADPS
SVO/GCPD/WBVR.W
SVO/GCPD/WBVR.B_ADPS
SVO/GCPD/WBVR.B
SVO/GCPD/WBVR.V_ADPS
SVO/GCPD/WBV

# Ingesting the Transmission curves into wsynphot format#

In [13]:
filter_index = pd.read_hdf('svo_filter_index.h5')

In [109]:
!mv filter_data.h5 filter_data.h5.old

In [21]:
pd.read_csv?

In [ ]:
with pd.HDFStore('filter_data.h5', 'w') as fh:
    for i, filter_data in tqdm_notebook(filter_index.iterrows(), total=len(filter_index)):
        fname = os.path.join('SVO', filter_data['Filter ID'])
        if 'SVO/INT/IPHAS' in fname:
            continue
        transmission_data = pd.read_csv(fname, names=['wavelength', 'transmission_lambda'], delim_whitespace=True, comment='#')
        if len(transmission_data)==0:
            continue
        transmission_data = transmission_data.astype(np.float64)
        wsynphot_filter_id = filter_data['Filter ID'].replace('.', '/').replace('-', '_')
        filter_index.loc[i, 'wsynphot_filter_id'] = wsynphot_filter_id
        fh[wsynphot_filter_id] = transmission_data

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2MASS'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '90prime'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '353_41'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '458_89'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '452_87'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '470_19'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '520_11'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '523_1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '527_12'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural 

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '577_12'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '586_21_2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '586_21'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natur

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '655_16'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '656_1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '656_2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural n

...

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '585'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '147'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '036'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming t

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '072'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '161'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '164'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming t

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '703'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '240'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '215'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming t

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '022'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '021'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '024'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming t

/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '0800'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '0890'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/wkerzend/miniconda/envs/casa_survivor/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '0990'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural namin

In [48]:
transmission_data.astype(np.float64)

,wavelength,transmission_lambda
0,1249.0,0.0
1,1250.0,1.0
2,1252.0,1.0
3,1254.0,1.0
4,1256.0,1.0
5,1258.0,1.0
6,1260.0,1.0
7,1262.0,1.0
8,1264.0,1.0
9,1266.0,1.0


In [9]:
fh = pd.HDFStore('filter_data.h5', 'a')

In [10]:
fh['index'] = filter_index

/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  """Entry point for launching an IPython kernel.
/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]

  """Entry point for launching an IPython kernel.
/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_items] [items->None]

  """Entry point for launching an IPython kernel.
/Users/wkerzend/anaconda3/envs/stark

In [12]:
fh.close()

In [16]:
import wsynphot
from astropy import units as u

In [44]:
x = wsynphot.FilterCurve.load_filter('Keck/NIRC2/Kp', wavelength_unit='angstrom')

In [43]:
(x.zp_ab_f_nu

<Quantity 3.631e-20 erg / (cm2 Hz s)>

In [27]:
filter_index[filter_index['Filter ID'].str.startswith('HST/WFC3_UVIS1.FQ619')]

,Filter ID,lambda_mean,lambda_eff,lambda_min,lambda_max,Weff,ZP (Jy),Obs. Facility,Instrument,Description
1797,HST/WFC3_UVIS1.FQ619N,6198.4,6198.2,6147,6253,60.9,3195.7,HST,WFC3_UVIS1,"HST, WFC3, UVIS1, FQ619N, CH4 6194"
